In [4]:
import pandas as pd
import numpy as np
from PIL import Image
import shutil
import os

In [5]:
df = pd.read_csv('../data/train.csv', sep=';')
df.head()

,object_id,name,description,group,img_name
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg
4,6339754,Сабля.,NaN,Оружие,2592073.jpg


In [6]:
df['group'].value_counts()

group
Редкие книги                 2025
Нумизматика                  1522
Прочие                       1511
ДПИ                          1496
Естественнонауч.коллекция    1479
Минералогия                  1455
Печатная продукция           1438
Техника                      1390
Скульптура                   1283
Фото, негативы               1227
Документы                    1190
Оружие                       1157
Графика                      1082
Живопись                     1057
Археология                   1033
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df, df['group'], test_size=0.05, random_state=42)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5)

In [9]:
print(len(x_train))
print(len(x_val))

19327
1018


In [10]:
cat = list(df['group'].unique())

In [27]:
for cat in examples.keys():
    os.mkdir(f'../data/yolo_best/train/{cat}')
    os.mkdir(f'../data/yolo_best/val/{cat}')

In [15]:
examples = dict()
for cat in x_test['group'].unique():
    ten_examples = x_test[x_test['group'] == cat].reset_index()
    examples[cat] = ten_examples

In [30]:
examples = dict()
for cat in x_train['group'].unique():
    ten_examples = x_train[x_train['group'] == cat].reset_index()
    examples[cat] = ten_examples

In [31]:
for cat in examples.keys():
    for i in range(len(examples[cat])):
        ex = examples[cat].iloc[i, :]
        shutil.copyfile(f"../data\\train\\{ex['object_id']}\\{ex['img_name']}", f"../data/yolo_best/train/{cat}/{ex['object_id']}_{ex['img_name']}")

In [ ]:
examples.keys()

In [16]:
for cat in examples.keys():
    for i in range(len(examples[cat])):
        ex = examples[cat].iloc[i, :]
        shutil.copyfile(f"../data\\train\\{ex['object_id']}\\{ex['img_name']}", f"../data/yolo/test/{cat}/{ex['img_name']}.jpg")

In [32]:
examples = dict()
for cat in x_val['group'].unique():
    ten_examples = x_val[x_val['group'] == cat].reset_index()
    examples[cat] = ten_examples

In [33]:
for cat in examples.keys():
    for i in range(len(examples[cat])):
        ex = examples[cat].iloc[i, :]
        shutil.copyfile(f"../data\\train\\{ex['object_id']}\\{ex['img_name']}", f"../data/yolo_best/val/{cat}/{ex['img_name']}")

In [1]:
from ultralytics import YOLO

# Load a pretrained YOLOV8 model
model = YOLO(r"runs/classify/train16/weights/best.pt")

In [2]:
model.val()

Ultralytics YOLOv8.1.47  Python-3.11.8 torch-2.4.0.dev20240405+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
YOLOv8x-cls summary (fused): 133 layers, 56142415 parameters, 0 gradients, 153.8 GFLOPs
train: E:\MachineLearningProjects\ml-practices\src\haccaton\data\yolo_best\train... found 19325 images in 15 classes  
val: E:\MachineLearningProjects\ml-practices\src\haccaton\data\yolo_best\val... found 1018 images in 15 classes  
test: None...


val: Scanning E:\MachineLearningProjects\ml-practices\src\haccaton\data\yolo_best\val... 1018 images, 0 corrupt: 100%|██████████| 1018/1018 [00:00<?, ?it/s]

val: WARNING  E:\MachineLearningProjects\ml-practices\src\haccaton\data\yolo_best\val\\31687598.jpg: corrupt JPEG restored and saved
val: WARNING  E:\MachineLearningProjects\ml-practices\src\haccaton\data\yolo_best\val\\9336514.jpg: corrupt JPEG restored and saved



               classes   top1_acc   top5_acc: 100%|██████████| 64/64 [00:06<00:00,  9.33it/s]


                   all      0.866      0.986
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020BA1E34E10>
curves: []
curves_results: []
fitness: 0.926326185464859
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8664047718048096, 'metrics/accuracy_top5': 0.9862475991249084, 'fitness': 0.926326185464859}
save_dir: WindowsPath('runs/classify/val')
speed: {'preprocess': 0.32598283763952945, 'inference': 3.6813257251132447, 'loss': 0.0009822470734302093, 'postprocess': 0.0}
task: 'classify'
top1: 0.8664047718048096
top5: 0.9862475991249084

In [12]:
lst = []
for c in cat:
    res = model.predict(source=f"../data/yolo_best/val/{c}", show=False, verbose=False)
    a = [r.probs.data for r in res]
    b = [c.cpu().numpy().tolist() for c in a]
    i = [d.index(max(d)) for d in b]
    lst.append([c, [res[0].names[j] for j in i]])

In [13]:
df1 = []
df2 = []
for element in lst:
    df1.extend(element[1])
    df2.extend([element[0]] * len(element[1]))
    

In [14]:
real = pd.DataFrame(list(zip(df1, df2)), columns=['predict', 'real'])

In [15]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

print(f1_score(df2, df1, average='weighted'))
print(accuracy_score(df2, df1))
print(recall_score(df2, df1, average='weighted'))
print(precision_score(df2, df1, average='weighted'))

0.8652481981669569
0.8664047151277013
0.8664047151277013
0.8661570245504827
